In [1]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0  53102      0  0:00:01  0:00:01 --:--:-- 53151
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   1103      0 --:--:-- --:--:-- --:--:--  1101
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   1112      0 --:--:-- --:--:-- --:--:--  1110


# Data Analysis Using Python's Pandas
Goal : Use the given dataset to rank population density of US states in 2010

### 1.1 Dataloading
* Given three dataset, each containing informatino about:
    1. Population
    2. Areas
    3. Abbreviations

In [2]:
import pandas as pd

In [3]:
#load each datasets and create dataframe
population = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [4]:
#preview each datasets
display(population.head())
display(areas.head())
display(abbrevs.head())

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [5]:
#check each dataframes' shape
print(population.shape)
print(areas.shape)
print(abbrevs.shape)

(2544, 4)
(52, 2)
(51, 2)


### 1.2 Data Joining
* Join state full name to population data

In [6]:
#merge 'population' and 'abbrevs' datasets, outer join, use 'state/region' from population and
#'abbreviation' from abbrevs to merge
merged = pd.merge(population, abbrevs, how='outer', \
                 left_on='state/region', right_on='abbreviation')

merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


Check null values in merged dataset

In [7]:
#check the merged dataset's shape first
merged.shape

(2544, 6)

In [8]:
#check if there are any missing values in merged dataset by column
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
abbreviation     True
dtype: bool

In [9]:
#find total number of null values in merged dataset by column
merged.isnull().sum()

state/region     0
ages             0
year             0
population      20
state           96
abbreviation    96
dtype: int64

In [10]:
#boolean masking rows whose values in 'population' column is null
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state,abbreviation
2448,PR,under18,1990,NaN,NaN,NaN
2449,PR,total,1990,NaN,NaN,NaN
2450,PR,total,1991,NaN,NaN,NaN
2451,PR,under18,1991,NaN,NaN,NaN
2452,PR,total,1993,NaN,NaN,NaN


### 1.2.1 We can check the following

1. 'state' name in rows where 'state/region' is PR are empty
2. Area dataset uses full state names, therefore we need to improve the problem above
3. We need to look at rows where state names empty

In [11]:
#find unique state/region whose state values are empty
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [12]:
#fill in Puerto Rico for PR, United States for USA
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

In [13]:
#check for missing values again
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
abbreviation     True
dtype: bool

### 1.2.2 Join Area Dataset

In [14]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,AL,52423.0
1,AL,total,2012,4817528.0,Alabama,AL,52423.0
2,AL,under18,2010,1130966.0,Alabama,AL,52423.0
3,AL,total,2010,4785570.0,Alabama,AL,52423.0
4,AL,under18,2011,1125763.0,Alabama,AL,52423.0


In [15]:
#check null values again
final.isnull().sum()

state/region      0
ages              0
year              0
population       20
state             0
abbreviation     96
area (sq. mi)    48
dtype: int64

There exist null values in 'area' column, where 'state' is 'United States'

In [23]:
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

Dealing with null values above
1. Area of United States is the sum of areas of states
2. We are only interested in population density of each states, so we can drop those columns

In [26]:
final.dropna(inplace=True)

In [27]:
#check years in final dataset
final['year'].unique()

array([2012, 2010, 2011, 2009, 2013, 2007, 2008, 2005, 2006, 2004, 2003,
       2001, 2002, 1999, 2000, 1998, 1997, 1996, 1995, 1994, 1993, 1992,
       1991, 1990])

In [29]:
#check ages in final dataset
final['ages'].unique()

array(['under18', 'total'], dtype=object)

## 1.3 Data Query

Query data of year 2010 and total ages

In [30]:
#year 2010 and total age
data_2010 = final.query("year==2010 & ages == 'total'")
data_2010.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,AL,52423.0
91,AK,total,2010,713868.0,Alaska,AK,656425.0
101,AZ,total,2010,6408790.0,Arizona,AZ,114006.0
189,AR,total,2010,2922280.0,Arkansas,AR,53182.0
197,CA,total,2010,37333601.0,California,CA,163707.0


Change index to states

In [31]:
data_2010.set_index('state', inplace=True)
data_2010.head()

,state/region,ages,year,population,abbreviation,area (sq. mi)
state,,,,,,
Alabama,AL,total,2010,4785570.0,AL,52423.0
Alaska,AK,total,2010,713868.0,AK,656425.0
Arizona,AZ,total,2010,6408790.0,AZ,114006.0
Arkansas,AR,total,2010,2922280.0,AR,53182.0
California,CA,total,2010,37333601.0,CA,163707.0


In [37]:
#total is 51, which is the sum of 50 states and District of Columbia
data_2010.index.unique()

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object', name='state')

## 1.4 Calculate Population Density

In [40]:
#population density is the population divided by areas
density = data_2010['population'] / data_2010['area (sq. mi)']
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

In [42]:
#sort population density in descending order to see which states/areas have the greatest population density
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
Massachusetts            621.815538
dtype: float64

In [43]:
#lowest population density
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

# Exercise : Find the average population density of years 2007, 2010, 2012

Average Population Density for 3 years = $\Large \frac{1}{3} \cdot \frac{2007 \enspace Population \enspace + \enspace 2010 \enspace Population \enspace + \enspace 2012 \enspace Population}{Area}$

### Solution 1

In [ ]:
#make separate dataframes
data_2007 = final.query("year == 2007 & ages == 'total'")
data_2010 = final.query("year == 2010 & ages == 'total'")
data_2012 = final.query("year == 2012 & ages == 'total'")

In [51]:
#replace index
data_2007.set_index('state', inplace=True)
data_2010.set_index('state', inplace=True)
data_2012.set_index('state', inplace=True)

In [54]:
#check index to make sure that all three dataframes have 51 unique states (including DC)
display(len(data_2007.index.unique()))
display(len(data_2010.index.unique()))
display(len(data_2012.index.unique()))

51

51

51

In [57]:
#check any missing values
display(data_2007.isnull().sum())
display(data_2010.isnull().sum())
display(data_2012.isnull().sum())

state/region     0
ages             0
year             0
population       0
abbreviation     0
area (sq. mi)    0
dtype: int64

state/region     0
ages             0
year             0
population       0
abbreviation     0
area (sq. mi)    0
dtype: int64

state/region     0
ages             0
year             0
population       0
abbreviation     0
area (sq. mi)    0
dtype: int64

In [59]:
density_2007 = data_2007['population'] / data_2007['area (sq. mi)']
density_2010 = data_2010['population'] / data_2010['area (sq. mi)']
density_2012 = data_2012['population'] / data_2012['area (sq. mi)']

In [63]:
average_density = (1/3) * (density_2007 + density_2010 + density_2012)
average_density.sort_values(ascending=False, inplace=True)

In [65]:
#top5
average_density.head()

state
District of Columbia    8887.039216
New Jersey              1006.968623
Rhode Island             681.831284
Connecticut              643.232624
Massachusetts            620.247118
dtype: float64

In [67]:
#last 5
average_density.tail()

state
South Dakota    10.554328
North Dakota     9.563396
Montana          6.711566
Wyoming          5.710346
Alaska           1.078811
dtype: float64

### Solution 2

In [82]:
combined = final.query("(year == 2007 or year == 2010 or year == 2012) & ages == 'total'")
combined.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
1,AL,total,2012,4817528.0,Alabama,AL,52423.0
3,AL,total,2010,4785570.0,Alabama,AL,52423.0
10,AL,total,2007,4672840.0,Alabama,AL,52423.0
82,AK,total,2007,680300.0,Alaska,AK,656425.0
91,AK,total,2010,713868.0,Alaska,AK,656425.0


In [94]:
import numpy as np
grouped_combined = combined.groupby('state').aggregate({'population':sum, 'area (sq. mi)':np.mean})
grouped_combined.head()

,population,area (sq. mi)
state,,
Alabama,14275938.0,52423.0
Alaska,2124475.0,656425.0
Arizona,19127620.0,114006.0
Arkansas,8720758.0,53182.0
California,111583790.0,163707.0


In [96]:
density_final = grouped_combined['population'] / grouped_combined['area (sq. mi)']
density_final = density_final/3

In [98]:
density_final.sort_values(ascending=False, inplace=True)
density_final.head()

state
District of Columbia    8887.039216
New Jersey              1006.968623
Rhode Island             681.831284
Connecticut              643.232624
Massachusetts            620.247118
dtype: float64

In [99]:
density_final.tail()

state
South Dakota    10.554328
North Dakota     9.563396
Montana          6.711566
Wyoming          5.710346
Alaska           1.078811
dtype: float64